In [3]:
# Installer les bibliothèques nécessaires

!pip install pydub SpeechRecognition pandas transformers openpyxl
!pip install transformers torchaudio

In [4]:
#Imortation des bibliothèques necessaires

import os
import pandas as pd
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer, pipeline
from pydub import AudioSegment
from google.colab import drive

# Monter Google Drive

drive.mount('/content/drive')

# Fonction de lecture fichier audio

def read_audio(file_path):
    audio = AudioSegment.from_file(file_path)
    return audio

# Charger le modèle et le tokenizer de Hugging Face

model_name = "facebook/wav2vec2-large-960h"
tokenizer = Wav2Vec2Tokenizer.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# Fonction pour convertir l'audio en texte

def audio_to_text(file_path):

    # Convertir le fichier audio en WAV si ce n'est pas déjà

    if not file_path.endswith('.wav'):
        audio_segment = AudioSegment.from_file(file_path)
        wav_path = file_path.rsplit('.', 1)[0] + '.wav'
        audio_segment.export(wav_path, format='wav')
        file_path = wav_path

    # Charger l'audio et effectuer la reconnaissance vocale

    waveform, sample_rate = torchaudio.load(file_path)
    input_values = tokenizer(waveform.squeeze().numpy(), return_tensors="pt", padding="longest").input_values

    # Effectuer la prédiction

    with torch.no_grad():
        logits = model(input_values).logits

    # Obtenir les indices des tokens

    predicted_ids = logits.argmax(dim=-1)

    # Convertir les indices en texte

    text = tokenizer.batch_decode(predicted_ids)[0]
    return text

# Dossier contenant les fichiers audio

audio_folder = '/content/drive/My Drive/dataset_audio'
results = []

# Lire les fichiers audio dans le dossier

for index, filename in enumerate(os.listdir(audio_folder)):
    if filename.endswith('.wav') or filename.endswith('.mp3'):
        file_path = os.path.join(audio_folder, filename)
        try:
            text = audio_to_text(file_path)

            results.append({
                'Numero': index + 1,
                'Texte': text,
            })
        except Exception as e:
            print(f"Erreur lors du traitement du fichier {filename}: {e}")

# Créer un DataFrame et l'exporter en Excel

df = pd.DataFrame(results)
df.to_excel('/content/drive/My Drive/Transforamtion_audio_texte.xlsx', index=False)

# Charger le modèle entraîné

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.load_state_dict(torch.load('/content/drive/My Drive/classification.pth'))
model.eval()

# Initialiser le tokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Charger le fichier Excel

input_file_path = '/content/drive/My Drive/Transforamtion_audio_texte.xlsx'  # Remplacez par le chemin de votre fichier Excel
output_file_path = '/content/drive/My Drive/Analyse_sentiments.xlsx' # Chemin pour le fichier de sortie

df = pd.read_excel(input_file_path)
texts = df['Texte']
order_numbers = df['Numero']

# Préparer les prédictions

predictions = []

with torch.no_grad():
    for text in texts:
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=128,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1).item()  # Obtenir la prédiction
        predictions.append("Positif" if prediction == 1 else "Négatif")  # 1 = Positif et 0 = Négatif

# Créer un DataFrame avec les résultats

df_results = pd.DataFrame({
    'Numero': order_numbers,
    'Texte': texts,
    'Sentiment prédit': predictions
})

# Sauvegarder les résultats dans un nouveau fichier Excel

df_results.to_excel(output_file_path, index=False)
print("Les résultats ont été sauvegardés avec succès dans le fichier Analyse_sentiments.xlsx")

print(df_results)

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-4-aab16864c7e4>:88: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We reco

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Les résultats ont été sauvegardés avec succès dans le fichier Analyse_sentiments.xlsx
   Numero                                              Texte Sentiment prédit
0       1  AATOSO N T     ONTO MO  OSH MO O A HONA M  PA ...          Négatif
1       2  NOTN A  HASTON PCOMONSHOPOTONNCHOO OIS POO TO ...          Négatif
2       3  PMAPO AN  PUL  P AUUN PU  PU V  PPU  A  A  O  ...          Positif
3       4  U OM TUOTEO MPU MO TEIMOO L P CONCANP ONNY HOM...          Négatif
4       5  OM TOOSOM  POO PO OM H FOTONO ONO PO O OMO P O...          Positif
